In [1]:
from quantum_logical.interaction import ConversionGainInteraction
import numpy as np
import itertools
from qutip import tensor, destroy
from quantum_logical.operators import (
    selective_destroy,
    reduce_to_two_qubit_subspace,
)
from weylchamber import c1c2c3

In [2]:
transmon_levels = 3
a_ge = selective_destroy(transmon_levels, 1, 0)
a_ef = selective_destroy(transmon_levels, 2, 1)
a_gf = selective_destroy(transmon_levels, 2, 0)

# NOTE items such as a_gf are excluded
# this would be a slower 5-wave mixing term
operators = [
    tensor(a_ge, a_ge),
    tensor(a_ge, a_ge.dag()),
    tensor(a_ef, a_ge),
    tensor(a_ef, a_ge.dag()),
    tensor(a_ge, a_ef),
    tensor(a_ge, a_ef.dag()),
    tensor(a_ef, a_ef),
    tensor(a_ef, a_ef.dag()),
]

In [3]:
# Coefficients' discrete set, including zero
coefficient_values = np.linspace(0, np.pi / 2, 5)
print(coefficient_values)


def generate_coefficient_operator_combinations(operators, coefficient_values):
    """
    Generator that yields lists of coefficients to be paired with operators.

    Parameters:
    operators (list): List of operator terms.
    coefficient_values (list): List of discrete coefficient values to consider, including zero.

    Yields:
    list, list: A list of coefficients and the list of corresponding operators.
    """
    # Use itertools.product to generate all possible combinations of coefficients
    for coeffs in itertools.product(coefficient_values, repeat=len(operators)):
        # Filter out the all-zero combination if needed
        if not all(c == 0 for c in coeffs):
            yield (list(coeffs))

[0.         0.39269908 0.78539816 1.17809725 1.57079633]


In [7]:
# Construct the Hamiltonian and corresponding unitary operator
import qutip

H = ConversionGainInteraction.from_coeff_ops_list(
    1 / np.sqrt(2) * np.ones(8), operators
)
U = H.construct_U(t=np.pi / 2)


u = reduce_to_two_qubit_subspace(U, [0, 2])
print(u.isunitary)
print(c1c2c3(u))
U

u = reduce_to_two_qubit_subspace(U, [0, 1])
print(u.isunitary)
print(c1c2c3(u))
U


u = reduce_to_two_qubit_subspace(U, [1, 2])
print(u.isunitary)
print(c1c2c3(u))
U

# from weylchamber import WeylChamber

# w = WeylChamber()
# w.scatter(*[0.79289322, 0.20710678, 0.0])
# w.plot()from weylchamber import WeylChamber

# w = WeylChamber()
# w.scatter(*[0.79289322, 0.20710678, 0.0])
# w.plot()

# TODO revist!!!
# this Hamiltonian is slightly different than the version in 02_cz_gf.ipynb
# in that version, we add identity terms on single qubit drives, then tensor them together
# which makes a difference
# not on the [0,2] gate, but on the [0,1] and [1,2] gates
# in other version these are identity
# but in this version they are not....
# I don't know which version is correct anymore :(

False
(0.5, 0.0, 0.0)
False
(0.5, 0.35355339, 0.35355339)


Quantum object: dims = [[3, 3], [3, 3]], shape = (9, 9), type = oper, isherm = False
Qobj data =
[[ 0.59857503+0.j         0.        +0.j        -0.40142497+0.j
   0.        +0.j         0.        -0.3978466j  0.        +0.j
  -0.40142497+0.j         0.        +0.j        -0.40142497+0.j       ]
 [ 0.        +0.j         0.19715007+0.j         0.        +0.j
   0.        -0.3978466j  0.        +0.j         0.        -0.3978466j
   0.        +0.j        -0.80284993+0.j         0.        +0.j       ]
 [-0.40142497+0.j         0.        +0.j         0.59857503+0.j
   0.        +0.j         0.        -0.3978466j  0.        +0.j
  -0.40142497+0.j         0.        +0.j        -0.40142497+0.j       ]
 [ 0.        +0.j         0.        -0.3978466j  0.        +0.j
   0.19715007+0.j         0.        +0.j        -0.80284993+0.j
   0.        +0.j         0.        -0.3978466j  0.        +0.j       ]
 [ 0.        -0.3978466j  0.        +0.j         0.        -0.3978466j
   0.        +0.j        

In [92]:
def is_valid_operator(U, target_subspace, identity_subspaces):
    """
    Determine if operator U is valid based on subspace criteria.

    Parameters:
    U (qutip.Qobj): The operator to be validated.
    target_subspace (list): The subspace where U should not be identity.
    identity_subspaces (list of lists): Subspaces where U should be identity.

    Returns:
    bool: True if U is valid, False otherwise.
    """
    # Check non-identity in the target subspace
    reduced_target = reduce_to_two_qubit_subspace(U, target_subspace)
    if not reduced_target.isunitary or np.allclose(c1c2c3(reduced_target), np.zeros(3)):
        return False

    # Check for identity in all other specified subspaces
    for iso in identity_subspaces:
        reduced_iso = reduce_to_two_qubit_subspace(U, iso)
        if not reduced_iso.isunitary or not np.allclose(
            c1c2c3(reduced_iso), np.zeros(3)
        ):
            return False

    # All checks passed
    return True


# Iterate over coefficient-operator combinations
for coeff_list in generate_coefficient_operator_combinations(
    operators, coefficient_values
):
    # Construct the Hamiltonian and corresponding unitary operator
    H = ConversionGainInteraction.from_coeff_ops_list(coeff_list, operators)
    U = H.construct_U(t=np.pi)

    # Validate the unitary operator against our criteria
    if is_valid_operator(
        U,
        target_subspace=[0, 2],
        identity_subspaces=[],  # , identity_subspaces=[[0, 1], [1, 2]]
    ):
        # Print information for valid operators only
        # try:
        coord = c1c2c3(reduce_to_two_qubit_subspace(U, [0, 2]))
        print(coeff_list, coord)
        # except (np.linalg.LinAlgError, ValueError):
        #     continue